#### LeetCode - SQL - #181

Refer [181. Employees Earning More Than Their Managers](https://leetcode.com/problems/employees-earning-more-than-their-managers/description/)

Write a solution to find the employees who earn more than their managers.

Return the result table in any order.

In [0]:
data = [[1, 'Joe', 70000, 3], [2, 'Henry', 80000, 4], [3, 'Sam', 60000, None], [4, 'Max', 90000, None]]
columns = ['id', 'name', 'salary', 'managerId']
schema = {'id':'Int64', 'name':'object', 'salary':'Int64', 'managerId':'Int64'}

In [0]:
import pandas

employees_pandas_dataframe = pandas.DataFrame(data=data, columns=columns).astype(schema)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
employees_dataframe = spark_context.createDataFrame(employees_pandas_dataframe)
employees_dataframe.printSchema()
employees_dataframe.show(5)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- managerId: long (nullable = true)

+---+-----+------+---------+
| id| name|salary|managerId|
+---+-----+------+---------+
|  1|  Joe| 70000|        3|
|  2|Henry| 80000|        4|
|  3|  Sam| 60000|     NULL|
|  4|  Max| 90000|     NULL|
+---+-----+------+---------+



In [0]:
from pyspark.sql.functions import col

employees_dataframe_null_filtered = employees_dataframe.filter(employees_dataframe.managerId.isNotNull())
employees_dataframe_null_filtered \
    .alias("root") \
    .join(employees_dataframe.alias("manager"), col("root.managerId") == col("manager.id"), "inner") \
    .filter(col("root.salary") > col("manager.salary")) \
    .select(col("root.name").alias("Employee")) \
    .show()

+--------+
|Employee|
+--------+
|     Joe|
+--------+

